#Prototipito de glovoz con

Primero glovoz nacio para ser una conexion bidireccional unicamente, solo de portuges español y español portuges, por lo tanto solo 2 conexiones (copielo y pegelo en su ordenador)


Para instalar depencencias

In [2]:
!pip install SpeechRecognition PyAudio gtts deep_translator

/bin/bash: line 1: clone: command not found
/bin/bash: line 1: cd: seamless_communication: No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for PyAudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyAudio
Failed to build PyAudio
ERROR: Could not build wheels for PyAudio, which is required to install pyproject.toml-based projects


Install seamless [(ver mas formas de integrar seamless )](https://colab.research.google.com/github/R3gm/InsightSolver-Colab/blob/main/SeamlessM4T.ipynb#scrollTo=HApRildjd1RP)

In [ ]:
!pip install fairseq2==0.1 pydub yt-dlp
!git clone https://github.com/facebookresearch/seamless_communication.git
%cd seamless_communication
!git checkout 01c1042841f9bce66902eb2c7512dbdd71d42112 # We will use a stable version; if you want to use the latest version, comment out this line.
!pip install .

Conexion "1"

In [ ]:
import socket
from tools.tools import HOST,FROMIDIOM,TOIDIOM, PORT,speak,translate,recordAudio
s = socket.socket()
s.connect((HOST,PORT))
while True:
	txt=recordAudio()
	txt=translate(txt,FROMIDIOM,TOIDIOM)
	s.send(txt.encode());
	if(txt == "Bye" or txt == "bye"):
		break
	rcvdData=str(s.recv(1024).decode())
	print ("N:",rcvdData)
	speak(rcvdData)

s.close()

Conexion "2"

In [ ]:
import socket
from tools.tools import HOST,FROMIDIOM,TOIDIOM, PORT,speak,translate,recordAudio
s = socket.socket()
s.bind(('', PORT))
s.listen(5)
c, addr = s.accept()
print("Socket Up and running with a connection from",addr)
try:
    while True:
        rcvdData = c.recv(1024).decode()
        txt=translate(rcvdData,FROMIDIOM,TOIDIOM)
        print( "S:",txt)
        speak(rcvdData)
        sendData =recordAudio()# input("N: ")
        c.send(sendData.encode())
        if(sendData == "Bye" or sendData == "bye"):
            break
except:
    c.close()
c.close()

Definicion de Funciones

In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
PORT=65426
#HOST='192.0.0.1'
HOST='0.0.0.0'

FROMIDIOM="es"
TOIDIOM="de"
def recordAudio():
    # Record Audio
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something!")
        audio = r.listen(source)

        # Speech recognition using Google Speech Recognition
        data = ""
        try:
            # Uses the default API key
            # To use another API key: `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
            data = r.recognize_google(audio)
            print("You said: " + data)
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            speak("I couldn't understand you")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
        data = data.lower()
        return data
def speak(audioString):
    print(audioString)
    tts = gTTS(text=audioString, lang='en')
    tts.save("audio.mp3")
    os.system("mpg123 audio.mp3")
try:
	def webTranslate(text,source,destiny):
		language_code_to_name = {"de":"deu","en":"eng","es":"spa","ru":"rus","fr":"fra","pt":"pot"}
		from seamless_communication.models.inference import Translator
		import torch
		translator = Translator(
				"seamlessM4T_large",
				"vocoder_36langs",
				torch.device("cuda:0")
		)
		translated_text, _, _ = translator.predict(text, "t2tt", language_code_to_name[destiny], src_lang=language_code_to_name[source])
		return translated_text
except:
  """
  anterior funcion para traduccion
  """
  def webTranslate(txt,writeIn,translateTo):
    """
    webTranslate(txt,writeIn,translateTo )
      - txt			  -text to trasnlate
      - writeIn		  -in which language is it written
      - translateTo	  -language to be translated
    rember language prefix
    en -> english
    es -> spanish
    ...
    """
    from deep_translator import GoogleTranslator
    translatedTxt = GoogleTranslator(source=writeIn, target=translateTo).translate(txt)
    return translatedTxt